In [28]:
import numpy as np
import json
import re
import sys

In [29]:
CTAGS_FNS = "./ctags.txt"
PERF_FOLDED = "perf_results/ais10.cnf-perf.log"

In [30]:
def match_main_line(line):
    children_pct_pattern = r"\s*(?P<children_pct>\d+\.\d+)%"
    self_pct_pattern = r"\s+(?P<self_pct>\d+\.\d+)%"
    command_pattern = r"\s+(?P<command>\S+)"
    sharedobject_pattern = r"\s+(?P<sharedobject>\S+)"
    symbol_pattern = r"\s+\[\.\]\s+(?P<symbol>\S+)"
    pattern = (
        "^" +
        children_pct_pattern +
        self_pct_pattern +
        command_pattern +
        sharedobject_pattern +
        symbol_pattern +
        "$"
    )

    match = re.match(pattern, line)
    if match:
        res = match.groupdict()
        res['children_pct'] = float(res['children_pct'])
        res['self_pct'] = float(res['self_pct'])
        return res
    else:
        return None

In [31]:
main_lines = []

with open(PERF_FOLDED, "r") as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        res = match_main_line(line)
        if res:
            main_lines.append(res)

main_lines.sort(key=lambda r: r["self_pct"], reverse=True)
main_lines

[{'children_pct': 56.32,
  'self_pct': 34.51,
  'command': 'miniC2D',
  'sharedobject': 'miniC2D',
  'symbol': 'set_literal'},
 {'children_pct': 17.73,
  'self_pct': 14.21,
  'command': 'miniC2D',
  'sharedobject': 'miniC2D',
  'symbol': 'construct_vtree_key'},
 {'children_pct': 12.35,
  'self_pct': 12.35,
  'command': 'miniC2D',
  'sharedobject': 'miniC2D',
  'symbol': 'remove_watched_clause'},
 {'children_pct': 17.34,
  'self_pct': 4.99,
  'command': 'miniC2D',
  'sharedobject': 'miniC2D',
  'symbol': 'move_watched_literal'},
 {'children_pct': 3.13,
  'self_pct': 3.13,
  'command': 'miniC2D',
  'sharedobject': 'libstdc++.so.6.0.30',
  'symbol': 'std::_Rb_tree_increment'},
 {'children_pct': 2.06,
  'self_pct': 2.06,
  'command': 'miniC2D',
  'sharedobject': 'miniC2D',
  'symbol': 'sat_is_subsumed_clause'},
 {'children_pct': 2.02,
  'self_pct': 2.02,
  'command': 'miniC2D',
  'sharedobject': 'miniC2D',
  'symbol': 'mark_vars_at_level'},
 {'children_pct': 2.0,
  'self_pct': 2.0,
  'comm

In [32]:
total_self_time = 0
for line in main_lines:
    total_self_time += line["self_pct"]

total_self_time

98.90000000000015

In [33]:
# redoing target functions with ctags

target_functions = []
with open(CTAGS_FNS, "r") as f:
    for line in f.readlines():
        line = line.strip()
        fn_name = line.split(maxsplit=1)[0]

        target_functions.append(fn_name)

target_functions = set(target_functions)
target_functions

{'allocate_manager_keys',
 'allocate_vtree_keys',
 'bits2bytes',
 'c2d_version',
 'clist_size',
 'compile_dispatcher',
 'compile_vtree',
 'compile_vtree_decomposed',
 'compile_vtree_leaf',
 'compile_vtree_shannon',
 'compile_with_literal',
 'construct_vtree_cache',
 'construct_vtree_key',
 'copy_key',
 'count_dispatcher',
 'count_vtree',
 'count_vtree_decomposed',
 'count_vtree_leaf',
 'count_vtree_shannon',
 'count_with_literal',
 'drop_cache_entry',
 'drop_vtree_cache_entries',
 'extended_file_name',
 'free_cache_entry',
 'free_manager_keys',
 'free_vtree_cache',
 'free_vtree_keys',
 'get_options',
 'init_options',
 'insert_cache',
 'lookup_cache',
 'main',
 'match_keys',
 'nnf_conjoin',
 'nnf_count_models',
 'nnf_count_nodes',
 'nnf_decomposable',
 'nnf_disjoin',
 'nnf_edge_count',
 'nnf_entails_cnf',
 'nnf_free',
 'nnf_literal2node',
 'nnf_load_from_file',
 'nnf_manager_extract_nnf',
 'nnf_manager_free',
 'nnf_manager_get_root',
 'nnf_manager_memory',
 'nnf_manager_new',
 'nnf_mana

In [34]:
# folded analysis

# PERF_FOLDED = "perf_results_folded.txt"
# PERF_FOLDED = "mini_test.txt"

fn_pcts = {}
for fn in target_functions:
    fn_pcts[fn] = 0
# fn_pcts = { "construct_vtree_key": 0 }

with open(PERF_FOLDED, "r") as f:
    lines = f.readlines()
    current_fn = None
    skip_section = False
    for i, line in enumerate(lines):
        line = line.strip()
        if len(line) == 0:
            break
        if line[0] == "#":
            continue
        res = match_main_line(line)
        if res:
            current_fn = res["symbol"]
            if current_fn not in target_functions:
                skip_section = True
                continue
            skip_section = False
            fn_pcts[current_fn] += res["self_pct"]
        else:
            if skip_section:
                continue
            pct, callstack = line.split(maxsplit=1)
            pct = float(pct.replace('%', ''))
            callstack = callstack.split(";")
            if callstack[0] == current_fn:
                callstack = callstack[1:]
            callstack = set(callstack)
            contains_other_targets = len(callstack.intersection(target_functions)) > 0
            if contains_other_targets:
                continue
            else:
                fn_pcts[current_fn] += pct
fn_pcts

{'insert_cache': 0.6900000000000001,
 'vtree_manager_new': 11.860000000000017,
 'pprint_bytes': 0,
 'vtree_shannon_var': 0.12,
 'copy_key': 0,
 'set_vtree_hashcode': 0,
 'sat_at_assertion_level': 0,
 'sat_clause2index': 0,
 'nnf_literal2node': 0,
 'print_error_and_exit': 0,
 'count_with_literal': 0,
 'print_vtree_cache_stats': 0.0,
 'sat_literal2var': 0.06,
 'clist_size': 0.59,
 'vtree_save': 0,
 'nnf_entails_cnf': 0,
 'free_cache_entry': 0,
 'nnf_save_to_file': 0,
 'sat_is_marked_var': 0.78,
 'construct_vtree_cache': 0,
 'sat_clause2literals': 0.18,
 'print_help': 0,
 'drop_vtree_cache_entries': 0.18,
 'drop_cache_entry': 0,
 'c2d_version': 0,
 'nnf_manager_get_root': 0,
 'free_vtree_cache': 0.62,
 'sat_unmark_var': 0.92,
 'sat_state_free': 0.12,
 'vtree_type': 0,
 'vtree_is_leaf': 0,
 'sat_is_irrelevant_var': 0.06,
 'free_vtree_keys': 0,
 'sat_clause_of_var': 0,
 'sat_var2index': 0,
 'sat_literal2index': 0,
 'sat_assert_unit_clauses': 0,
 'sat_var2nliteral': 0.06,
 'nnf_count_nodes':

In [35]:
total_pct = 0
for fn, pct in fn_pcts.items():
    total_pct += pct

print(total_pct)
sorted_fn_pcts = sorted(list(fn_pcts.items()), key=lambda x: x[1], reverse=True)
if total_pct > 100:
    # normalize
    for i, (fn, pct) in enumerate(sorted_fn_pcts):
        norm_pct = pct / total_pct * 100
        sorted_fn_pcts[i] = (fn, norm_pct)
sorted_fn_pcts

99.11000000000003


[('sat_decide_literal', 41.54),
 ('sat_assert_clause', 15.01),
 ('construct_vtree_key', 14.21),
 ('vtree_manager_new', 11.860000000000017),
 ('sat_is_implied_literal', 3.440000000000001),
 ('sat_is_subsumed_clause', 2.06),
 ('vtree_is_shannon_node', 1.58),
 ('sat_mark_var', 1.53),
 ('lookup_cache', 1.26),
 ('sat_unmark_var', 0.92),
 ('sat_is_marked_var', 0.78),
 ('compile_dispatcher', 0.72),
 ('insert_cache', 0.6900000000000001),
 ('free_vtree_cache', 0.62),
 ('clist_size', 0.59),
 ('compile_vtree_shannon', 0.55),
 ('sat_var2pliteral', 0.19),
 ('sat_is_instantiated_var', 0.19),
 ('sat_clause2literals', 0.18),
 ('drop_vtree_cache_entries', 0.18),
 ('free_manager_keys', 0.18),
 ('main', 0.18),
 ('vtree_shannon_var', 0.12),
 ('sat_state_free', 0.12),
 ('sat_mark_clause', 0.11),
 ('sat_literal2var', 0.06),
 ('sat_is_irrelevant_var', 0.06),
 ('sat_var2nliteral', 0.06),
 ('sat_unmark_clause', 0.06),
 ('var2nnf', 0.06),
 ('pprint_bytes', 0),
 ('copy_key', 0),
 ('set_vtree_hashcode', 0),
 ('sa

In [36]:
# alternate method
# iterate over the callstack
# find the topmost function which is in target functions -> add to that fn

# target_functions_no_compile = set(filter(lambda x: not x.startswith("compile_") and not "mark" in x, target_functions))
target_functions_no_compile = target_functions

fn_pcts = {}
for fn in target_functions_no_compile:
    fn_pcts[fn] = 0

with open(PERF_FOLDED, "r") as f:
    lines = f.readlines()
    skip_section = False
    current_fn = None
    for i, line in enumerate(lines):
        line = line.strip()
        if len(line) == 0:
            break
        if line[0] == "#":
            continue
        res = match_main_line(line)
        if res:
            current_fn = res["symbol"]
            if current_fn not in target_functions_no_compile:
                skip_section = True
                continue
            skip_section = False
        else:
            if skip_section:
                continue
            pct, callstack = line.split(maxsplit=1)
            pct = float(pct.replace('%', ''))
            callstack = callstack.split(";")
            for fn in reversed(callstack):
                if fn in target_functions_no_compile:
                    if fn == current_fn:
                        fn_pcts[current_fn] += pct
                    break
fn_pcts = sorted(list(fn_pcts.items()), key=lambda x: x[1], reverse=True)
fn_pcts

[('sat_decide_literal', 41.53999999999999),
 ('sat_assert_clause', 15.01),
 ('construct_vtree_key', 14.050000000000027),
 ('vtree_manager_new', 11.860000000000017),
 ('compile_vtree_shannon', 4.2200000000000015),
 ('sat_is_subsumed_clause', 2.0400000000000014),
 ('sat_mark_var', 1.53),
 ('lookup_cache', 1.2600000000000007),
 ('sat_is_implied_literal', 1.2600000000000007),
 ('sat_unmark_var', 0.9099999999999999),
 ('sat_is_marked_var', 0.78),
 ('compile_dispatcher', 0.7700000000000002),
 ('vtree_is_shannon_node', 0.7700000000000002),
 ('free_vtree_cache', 0.62),
 ('clist_size', 0.59),
 ('insert_cache', 0.18),
 ('sat_clause2literals', 0.18),
 ('drop_vtree_cache_entries', 0.18),
 ('sat_var2pliteral', 0.18),
 ('main', 0.18),
 ('sat_is_instantiated_var', 0.18),
 ('vtree_shannon_var', 0.12),
 ('sat_state_free', 0.12),
 ('compile_vtree_decomposed', 0.12),
 ('sat_mark_clause', 0.11),
 ('sat_literal2var', 0.06),
 ('sat_is_irrelevant_var', 0.06),
 ('sat_var2nliteral', 0.06),
 ('sat_unmark_clause

In [37]:
print(sum(fn_pcts.values()))

AttributeError: 'list' object has no attribute 'values'